In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import pickle
import re
from sklearn.model_selection import train_test_split

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif',
             'serif':['Times New Roman'],
             'size': 12,
            })

# PCS-Net vs. Paired Models, with varying number of avg. pairwise comparisons

## PCS-Net

In [ ]:
def compute_ranking_accuracy(df_, margin=0):
    # Split in non-ties and ties
    df_nonties = df_[df_.label_r != 0]
    df_ties = df_[df_.label_r == 0]

    # Split non ties per their outcome (left and right)
    df_nonties_left = df_nonties[(df_nonties.label_r == -1)]
    df_nonties_right = df_nonties[(df_nonties.label_r == 1)]

    # Non-ties accuracy
    correct_left = ((df_nonties.label_r == -1) & (df_nonties.rank_left - df_nonties.rank_right > margin)).sum()
    correct_right = ((df_nonties.label_r == 1) & (df_nonties.rank_right - df_nonties.rank_left > margin)).sum()

    nontie_left_accuracy = correct_left / (df_nonties.label_r == -1).sum()
    nontie_right_accuracy = correct_right / (df_nonties.label_r == 1).sum()
    nontie_accuracy = (correct_left + correct_right ) / df_nonties.shape[0]
    
    # Ties accuracy
    tie_accuracy = (abs(df_ties.rank_left - df_ties.rank_right) < margin).sum() / df_ties.shape[0]

    # Overall accuracy
    overall_accuracy = X_test[((df_.label_r == -1) & (df_.rank_left - df_.rank_right > margin)) |
                              ((df_.label_r ==  1) & (df_.rank_right - df_.rank_left > margin)) |
                              ((df_.label_r ==  0) & (abs(df_.rank_left - df_.rank_right) < margin))].shape[0] / df_.shape[0]
    
    return nontie_left_accuracy, nontie_right_accuracy, nontie_accuracy, tie_accuracy, overall_accuracy

In [ ]:
def compute_ranking_accuracy_nomargin(df_,):
    # Split in non-ties and ties
    df_nonties = df_[df_.label_r != 0]

    # Split non ties per their outcome (left and right)
    df_nonties_left = df_nonties[(df_nonties.label_r == -1)]
    df_nonties_right = df_nonties[(df_nonties.label_r == 1)]

    # Non-ties accuracy
    correct_left = ((df_nonties.label_r == -1) & (df_nonties.rank_left - df_nonties.rank_right > 0)).sum()
    correct_right = ((df_nonties.label_r == 1) & (df_nonties.rank_right - df_nonties.rank_left > 0)).sum()

    nontie_left_accuracy = correct_left / (df_nonties.label_r == -1).sum()
    nontie_right_accuracy = correct_right / (df_nonties.label_r == 1).sum()
    nontie_accuracy = (correct_left + correct_right ) / df_nonties.shape[0]

    return nontie_left_accuracy, nontie_right_accuracy, nontie_accuracy

In [ ]:
def compute_ranking_distance(df_):
    # Split in non-ties and ties
    df_nonties = df_[df_.label_r != 0]
    df_ties = df_[df_.label_r == 0]

    # Distance between non-ties
    avg_dist_nonties = abs(df_nonties.rank_left - df_nonties.rank_right).mean()
    
    # Distance between ties
    avg_dist_ties = abs(df_ties.rank_left - df_ties.rank_right).mean()
    
    return avg_dist_nonties, avg_dist_ties

In [ ]:
def compute_classification_accuracy(df_):
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
        
    col1_values = df_['logits_l'].values
    col2_values = df_['logits_0'].values
    col3_values = df_['logits_r'].values
        
    probabilities = np.apply_along_axis(softmax, axis=1, arr=np.column_stack((col1_values, col2_values, col3_values)))
    max_indices = np.argmax(probabilities, axis=1)
    # Convert the probabilities back to a DataFrame with appropriate column names
    softmax_df = pd.DataFrame(probabilities, columns=['softmax_logit_l', 'softmax_logit_0', 'softmax_logit_r'])
    max_index_df = pd.DataFrame({'class_predicted': max_indices})
    # Concatenate the new DataFrame with the original DataFrame if needed
    result_df = pd.concat([df.reset_index(drop=True), softmax_df, max_index_df], axis=1,)

    all_accuracy = (result_df.class_predicted == result_df.label_c).sum() / result_df.shape[0]
    tie_accuracy = (result_df[result_df.label_c == 1].class_predicted == result_df[result_df.label_c == 1].label_c).sum() / result_df.shape[0]
    nontie_accuracy = (result_df[result_df.label_c != 1].class_predicted == result_df[result_df.label_c != 1].label_c).sum() / result_df.shape[0]
    
    return all_accuracy, tie_accuracy, nontie_accuracy

### Available results

In [ ]:
model_results = glob('../outputs/saved/synthetic-*.pkl')

In [ ]:
model_data = glob('../data/comparisons_synthetic_pc*.p')

In [ ]:
pd.read_pickle(model_data[0]).individual_id.unique().shape

### Process results

In [ ]:
compiled_results = []
for model_result in model_results:
    df = pd.read_pickle(model_result)
    avg_pc = float(os.path.basename(model_result).replace('synthetic-', '').replace('.pt_results.pkl', ''))
    orig
    seed = 30
    margin_= 1
    print('Avg. Pairwise Comparisons:', avg_pc, '-->', model_result)

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=seed)
    X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)
    # print('\tTrain:     ', X_train.shape)
    # print('\tValidation:', X_val.shape) 
    # print('\tTest:      ', X_test.shape)

    # Ranking sub-network
    nontie_left_accuracy, nontie_right_accuracy, nontie_accuracy, tie_accuracy, overall_accuracy = compute_ranking_accuracy(X_test, margin=margin_)

    # Ranking sub-network, without any margin on accuracy
    nontie_left_accuracy_nomargin, nontie_right_accuracy_nomargin, nontie_accuracy_nomargin = compute_ranking_accuracy_nomargin(X_test)

    # Classification sub-network
    c_all_accuracy, c_tie_accuracy, c_nontie_accuracy = compute_classification_accuracy(X_test)

    # Rank difference
    avg_dist_nonties, avg_dist_ties = compute_ranking_distance(X_test)
    
    # Compile results
    result = {
        'avg_pc': avg_pc,
        'seed': seed,
        # Ranking, with margins
        'ranking_acc': overall_accuracy,
        'ranking_acc_nonties': nontie_accuracy,
        'ranking_acc_ties': tie_accuracy,
        'ranking_acc_left': nontie_left_accuracy,
        'ranking_acc_right': nontie_right_accuracy,
        # Ranking, without margins
        'ranking_acc_nonties_nomargin': nontie_accuracy_nomargin,
        'ranking_acc_left_nomargin': nontie_left_accuracy_nomargin,
        'ranking_acc_right_nomargin': nontie_right_accuracy_nomargin,
        # Classification
        'classification_acc': c_all_accuracy,
        'classification_acc_nonties': c_nontie_accuracy,
        'classification_acc_ties': c_tie_accuracy,
        # Rank difference
        'avg_dist_nonties': avg_dist_nonties,
        'avg_dist_ties': avg_dist_ties,
    }
    compiled_results.append(result)
    
results_df = pd.DataFrame(compiled_results)

In [ ]:
results_df_pcs = results_df.sort_values(by=['avg_pc'])
results_df_pcs

## TrueSkill

In [ ]:
import itertools
import json
import math
import numpy as np
import os
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import trueskill as ts

In [ ]:
def compute_probabilities(team1, team2):
    BETA = ts.BETA
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts_ = ts.global_env()
    return ts_.cdf(delta_mu / denom), 1 - ts_.cdf(delta_mu / denom)

In [ ]:
def compute_accuracy(df):
    accuracy = []
    
    for i, row in df.iterrows():
        p_win, p_los = compute_probabilities([scores[row.Winner]], [scores[row.Loser]])
    
        if row.score == -1 or row.score == 1:
            accuracy.append(int(p_win > p_los))
    
    return accuracy

### Run

In [ ]:
model_results = glob('../data/comparisons_synthetic_pc*.p')

In [ ]:
results = []
for df_name  in model_results:
    comparisons_ = pickle.load(open(df_name, 'rb'))
    avg_pc = os.path.basename(df_name).replace('comparisons_synthetic_pc', '').replace('.p', '')
    seed = 30
    print('Avg comparisons:', avg_pc)

    comparisons_['image_l'] = comparisons_['scene_i']
    comparisons_['image_r'] = comparisons_['scene_j']
    comparisons_['Winner'] = -1
    comparisons_['Loser'] = -1
    comparisons_['Tie'] = 0
    
    for i, row in comparisons_.iterrows():
        l_item = row.image_l
        r_item = row.image_r
        
        if row.score == 1:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == -1:
            comparisons_.loc[i, 'Winner'] = l_item
            comparisons_.loc[i, 'Loser'] = r_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == 0:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 1

    
    unique_images = pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K'))

    # Split data in Train, Validation & Test
    X_train, X_test = train_test_split(comparisons_, test_size=0.2, random_state=seed)
    X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)
    
    # Initialize TrueSkill scores
    scores = {}

    for image in unique_images:
        scores[image] = ts.Rating()

    # Compute scores based on comparisons
    for i, row in X_train.iterrows():
        # Define the players in this round
        player1 = scores[row['image_l']]
        player2 = scores[row['image_r']]
        
        # Process match
        if row['score'] == -1:
            score = [0, 1]
        elif row['score'] == 0:
            score = [0, 0]
        elif row['score'] == 1:
            score = [1, 0]
        
        [player1], [player2] = ts.rate([[player1], [player2]], ranks=score)
    
        # Update scores
        scores[row['image_l']] = player1
        scores[row['image_r']] = player2

    scores_df = pd.DataFrame(scores).T
    scores_df.columns = ['score', 'sigma']
    scores_df['image_path'] = scores_df.index
    scores_df['image'] = scores_df.index
    #scores_df['image_path'] = scores_df['image_path'].apply(lambda x: os.path.join('images','berlin', x + '.jpg' ))

    # Train
    # log_loss_train = compute_logloss(X_train[X_train.score != 0])
    accuracy_train = compute_accuracy(X_train[X_train.score != 0])

    # Test
    # log_loss_test = compute_logloss(X_test[X_test.score != 0])
    accuracy_test = compute_accuracy(X_test[X_test.score != 0])

    # Compile results
    result = {
        'avg_pc': avg_pc,
        'n_pc': comparisons_.shape[0],
        'model': 'trueskill',
        'train_accuracy': np.mean(accuracy_train),
        'test_accuracy': np.mean(accuracy_test),
        'seed': seed
    }
    results.append(result)

In [ ]:
results_df_ts = pd.DataFrame(results)
results_df_ts.avg_pc = results_df_ts.avg_pc.astype(float)
results_df_ts = results_df_ts.sort_values(by=['avg_pc'])
results_df_ts

## Elo

In [ ]:
class OriginalELo(object):
    def __init__(self, k_factor, elo_width, starting_elo):
        self.k_factor = k_factor
        self.elo_width = elo_width
        self.starting_elo = starting_elo
        self.items = set()
        self.items_elo = dict()
        
    def initialize_items(self, items):
        """Initialize the items available to `items`."""
        self.items = set(items)
    
    def initialize_elos(self, ):
        """Set the initial starting elo for all available items."""
        for item in self.items:
            self.items_elo[item] = self.starting_elo
    
    def expected_result(self, elo_a, elo_b):
        """Expected probability of item with elo_a winning vs. item with elo_b."""
        
        expect_a = 1.0/(1+10**((elo_b - elo_a)/self.elo_width))
        return expect_a
    
    def update_elo(self, winner_elo, loser_elo, tie=False):
        """Update elo for the winning item and losing item."""
        
        R = 1
        if tie:
            R = .5
        
        expected_win = self.expected_result(winner_elo, loser_elo)  
        change_in_elo = self.k_factor * (R-expected_win)
        
        winner_elo += change_in_elo
        loser_elo -= change_in_elo
        return winner_elo, loser_elo
    
    def add_comparison(self, w_item, l_item, tie=False):
        """Process comparison between winning item and losing item."""
        current_winner_elo = self.items_elo[w_item]
        current_loser_elo = self.items_elo[l_item]
        
        updated_winner_elo, updated_loser_elo = self.update_elo(current_winner_elo, current_loser_elo, tie=tie)
        
        self.items_elo[w_item] = updated_winner_elo
        self.items_elo[l_item] = updated_loser_elo
        

In [ ]:
def compute_probabilities(elo_a, elo_b, allow_ties=False):
    """
    Expected probabilities of winning, drawing, or losing.
    Reference for draws formula: `Mathematical Model of Ranking Accuracy and Popularity Promotion`
    https://www.researchgate.net/publication/309662241_Mathematical_Model_of_Ranking_Accuracy_and_Popularity_Promotion
    """
      
    p_win = 1. / (1+10**((-elo_a + elo_b)/elo_width))
    p_los = 1. / (1+10**((elo_a - elo_b)/elo_width))
    
    if allow_ties:
        p_tie = (1 / (np.sqrt(2 * np.pi) * np.e)) * np.exp(-1 * (( (elo_a-elo_b)/(elo_width/2) )**2) / (2*np.e**2))
        p_win = p_win - 0.5 * p_tie  
        p_los = p_los - 0.5 * p_tie
        
        return  p_win, p_los, p_tie

    return p_win, p_los
    
def compute_logloss(df):
    log_loss = []
    for i, row in df.iterrows():
        # p_win, p_los, p_tie = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])
        p_win, p_los = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])

        if row.score == -1 or row.score == 1:
            log_loss.append(np.log(p_win))
        else:
            log_loss.append(np.log(p_tie))
    
    return log_loss
    
def compute_accuracy(df):
    accuracy = []
    
    for i, row in df.iterrows():
        # p_win, p_los, p_tie = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])
        p_win, p_los = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])

        if row.score == -1 or row.score == 1:
            accuracy.append(int(p_win > p_los))
        else:
            accuracy.append(int(p_win > p_los))
    
    return accuracy

### Run

In [ ]:
model_results = glob('../data/comparisons_synthetic_pc*.p')

In [ ]:
results = []
for df_name  in model_results:
    comparisons_ = pickle.load(open(df_name, 'rb'))
    avg_pc = os.path.basename(df_name).replace('comparisons_synthetic_pc', '').replace('.p', '')
    seed = 30
    print('Avg comparisons:', avg_pc)

    comparisons_['image_l'] = comparisons_['scene_i']
    comparisons_['image_r'] = comparisons_['scene_j']
    comparisons_['Winner'] = -1
    comparisons_['Loser'] = -1
    comparisons_['Tie'] = 0
    
    for i, row in comparisons_.iterrows():
        l_item = row.image_l
        r_item = row.image_r
        
        if row.score == 1:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == -1:
            comparisons_.loc[i, 'Winner'] = l_item
            comparisons_.loc[i, 'Loser'] = r_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == 0:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 1

    
    unique_images = pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K'))

    # Split data in Train, Validation & Test
    X_train, X_test = train_test_split(comparisons_, test_size=0.2, random_state=seed)
    X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

    # ELO
    starting_elo = 1500
    elo_width = 400
    k_factor = 32
    
    elo = OriginalELo(k_factor=k_factor, 
                      elo_width=elo_width, 
                      starting_elo=starting_elo)
    elo.initialize_items(list(comparisons_.Winner.values) + list(comparisons_.Loser.values))
    elo.initialize_elos()

    for i, row in X_train.iterrows():
        w_item = row.Winner
        l_item = row.Loser 
        tie = True if row.Tie else False
        
        elo.add_comparison(w_item, l_item, tie=tie)    
    scores = []

    for item, item_elo in elo.items_elo.items():
        scores.append({
            'score': item_elo,
            'image': item,
            'image_path': os.path.join('images','berlin', item + '.jpg' )
        })
    scores_df = pd.DataFrame(scores).set_index('image', drop=False)


    accuracy_test = compute_accuracy(X_test[X_test.score != 0])
    #
    # Compile results
    result = {
        'avg_pc': avg_pc,
        'n_pc': comparisons_.shape[0],
        'model': 'elo',
        'train_accuracy': np.mean(accuracy_train),
        'test_accuracy': np.mean(accuracy_test),
        'seed': seed
    }
    results.append(result)

In [ ]:
results_df_elo = pd.DataFrame(results)
results_df_elo.avg_pc = results_df_elo.avg_pc.astype(float)
results_df_elo = results_df_elo.sort_values(by=['avg_pc'])
results_df_elo

## Gaussian Process

In [ ]:
import kickscore as ks

In [ ]:
def compute_accuracy(df, t_):
    accuracy = []
    
    for i, row in df.iterrows():
        try:
            p_win, p_tie, p_los = model.probabilities([row.Winner], [row.Loser], t=t_)

            if row.score == -1 or row.score == 1:
                if p_win > p_los and p_win > p_tie:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
            else:  
                if p_tie > p_los and p_tie > p_win:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
        except KeyError:
            continue
    return accuracy

### Run

In [ ]:
model_results = glob('../data/comparisons_synthetic_pc*.p')

In [ ]:
results = []
for df_name  in model_results:
    comparisons_ = pickle.load(open(df_name, 'rb'))
    avg_pc = os.path.basename(df_name).replace('comparisons_synthetic_pc', '').replace('.p', '')
    seed = 30
    print('Avg comparisons:', avg_pc)

    comparisons_['image_l'] = comparisons_['scene_i']
    comparisons_['image_r'] = comparisons_['scene_j']
    comparisons_['Winner'] = -1
    comparisons_['Loser'] = -1
    comparisons_['Tie'] = 0
    
    for i, row in comparisons_.iterrows():
        l_item = row.image_l
        r_item = row.image_r
        
        if row.score == 1:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == -1:
            comparisons_.loc[i, 'Winner'] = l_item
            comparisons_.loc[i, 'Loser'] = r_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == 0:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 1

    
    unique_images = pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K'))

    # Split data in Train, Validation & Test
    X_train, X_test = train_test_split(comparisons_, test_size=0.2, random_state=seed)
    X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

    # ========================================================================================================== #
    # GAUSSIAN PROCESS
    images = set()
    observations = list()
    
    for i, row in X_train.reset_index().iterrows():
        t = i
        images.add(row.image_l)
        images.add(row.image_r)
        
        if row.score == -1:
            observations.append({
                    'winners': [row.image_l],
                    'losers': [row.image_r],
                    #'tie': False,
                    't': t,
                })
        if row.score == 0:
            observations.append({
                    'winners': [row.image_l],
                    'losers': [row.image_r],
                    'tie': True,
                    't': t,
                })
        if row.score == 1:
            observations.append({
                    'winners': [row.image_r],
                    'losers': [row.image_l],
                    #'tie': False,
                    't': t,
                })

    model = ks.TernaryModel(margin=0.2)
    kernel = (ks.kernel.Constant(var=0.03))
    for image in images:
        model.add_item(image, kernel=kernel)
    for obs in observations:
        model.observe(**obs)

    converged = model.fit()
    if converged:
        print("Model has converged.")

    ts = [comparisons_.shape[0] ]  # Point in time at which you want to make the prediction.
    res = dict()  # Contains predicted score.
    
    scores = []
    for name, item in model.item.items():
        means, var = item.predict(ts)
        scores += [[name, means[0], var[0]]]
    
    scores = pd.DataFrame(scores, columns=['image', 'score', 'var'])
    
    scores_df = scores.sort_values(by='image').reset_index(drop=True)
    #for rank, (name, score) in enumerate(sorted(res.items(), key=lambda x: x[1], reverse=True), start=1):
    #    print(f"rank {rank}: {name} (score: {score:.2f}")
    scores_df = scores_df.set_index('image', drop=False)
    scores_df.index.name = None
    scores_df['image_path'] = scores_df['image']
    scores_df['image_path'] = scores_df['image_path'].apply(lambda x: os.path.join('images','berlin', str(x) + '.jpg' ))

    accuracy_train = compute_accuracy(X_train[X_train.score != 0], comparisons_.shape[0])
    accuracy_test = compute_accuracy(X_test[X_test.score != 0], comparisons_.shape[0])

    # ========================================================================================================== #
    
    # Compile results
    result = {
        'avg_pc': avg_pc,
        'n_pc': comparisons_.shape[0],
        'model': 'gaussian_process',
        'train_accuracy': np.mean(accuracy_train),
        'test_accuracy': np.mean(accuracy_test),
        'seed': seed
    }
    results.append(result)

In [ ]:
results_df_gp = pd.DataFrame(results)
results_df_gp.avg_pc = results_df_gp.avg_pc.astype(float)
results_df_gp = results_df_gp.sort_values(by=['avg_pc'])
results_df_gp

## Rank Centrality

In [ ]:
import choix

In [ ]:
def compute_accuracy(df, params):
    accuracy = []
    
    for i, row in df.iterrows():
        if row.score == -1:
            p_win, p_los = choix.probabilities([int(row.image_l), int(row.image_r)], params)
        elif row.score == 1:
            p_win, p_los = choix.probabilities([int(row.image_r), int(row.image_l)], params)

        if row.score == -1 or row.score == 1:
            if p_win > p_los:
                accuracy.append(1) 
            else:
                accuracy.append(0)  

    return accuracy

### Run

In [ ]:
model_results = glob('../data/comparisons_synthetic_pc*.p')

In [ ]:
results = []
for df_name in model_results:
    comparisons_ = pickle.load(open(df_name, 'rb'))
    avg_pc = os.path.basename(df_name).replace('comparisons_synthetic_pc', '').replace('.p', '')
    seed = 30
    print('Avg comparisons:', avg_pc)

    comparisons_['image_l'] = comparisons_['scene_i']
    comparisons_['image_r'] = comparisons_['scene_j']
    comparisons_['Winner'] = -1
    comparisons_['Loser'] = -1
    comparisons_['Tie'] = 0
    
    for i, row in comparisons_.iterrows():
        l_item = row.image_l
        r_item = row.image_r
        
        if row.score == 1:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == -1:
            comparisons_.loc[i, 'Winner'] = l_item
            comparisons_.loc[i, 'Loser'] = r_item
            comparisons_.loc[i, 'Tie'] = 0
        elif row.score == 0:
            comparisons_.loc[i, 'Winner'] = r_item
            comparisons_.loc[i, 'Loser'] = l_item
            comparisons_.loc[i, 'Tie'] = 1

    # ========================================================================================================== #
    # RANK CENTRALITY
    n_items = len(pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K')))
    images = pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K'))
    images_dict = {}
    images_dict_rev = {}
    for i, image_id in enumerate(images):
        images_dict[image_id] = i
        images_dict_rev[i] = image_id
    comparisons_=comparisons_.replace({"image_l": images_dict})
    comparisons_=comparisons_.replace({"image_r": images_dict})
    comparisons_=comparisons_.replace({"Winner": images_dict})
    comparisons_=comparisons_.replace({"Loser": images_dict})

    # Split data in Train, Validation & Test
    X_train, X_test = train_test_split(comparisons_, test_size=0.2, random_state=seed)
    X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

    data = []
    for i, row in X_train.iterrows():
        if not row.Tie:
            data.append((int(row.Winner), int(row.Loser)))
    
        if row.Tie:
            data.append((row.Winner, row.Loser))
            data.append((row.Loser, row.Winner))
    
    params_rc = choix.rank_centrality(n_items, data, alpha=1e-4)
    
    accuracy_train = compute_accuracy(X_train[X_train.score != 0], params_rc)
    accuracy_test = compute_accuracy(X_test[X_test.score != 0], params_rc)
    
    # ========================================================================================================== #
    
    # Compile results
    result = {
        'avg_pc': avg_pc,
        'n_pc': comparisons_.shape[0],
        'model': 'rank_centrality',
        'train_accuracy': np.mean(accuracy_train),
        'test_accuracy': np.mean(accuracy_test),
        'seed': seed
    }

    results.append(result)

In [ ]:
results_df_rc = pd.DataFrame(results)
results_df_rc.avg_pc = results_df_rc.avg_pc.astype(float)
results_df_rc = results_df_rc.sort_values(by=['avg_pc'])
results_df_rc

## Plot Results

In [ ]:
plt.figure(figsize=(5,3))
#plt.plot(df.avg_pc, df.ranking_acc, 'k', label='With ties')
plt.plot(results_df_pcs.avg_pc, results_df_pcs.ranking_acc_nonties_nomargin, 'b', label='PCS-Net', )
plt.plot(results_df_ts.avg_pc, results_df_ts.test_accuracy, 'g:', label='TrueSkill', )
plt.plot(results_df_elo.avg_pc, results_df_elo.test_accuracy, 'r:', label='Elo', )
plt.plot(results_df_gp.avg_pc, results_df_gp.test_accuracy, 'c:', label='Gaussian Process', )
plt.plot(results_df_rc.avg_pc, results_df_rc.test_accuracy, 'y:', label='Rank Centrality', )
plt.xlabel('Avg. Comparisons per Image')
plt.ylabel('Accuracy (%)')
plt.ylim(0.5, 1)
plt.xlim(0, 40)
plt.legend()
plt.show()

# PCS-Net: Ranks

## PCS-Net

In [ ]:
def compute_ranking_accuracy(df_, margin=0):
    # Split in non-ties and ties
    df_nonties = df_[df_.label_r != 0]
    df_ties = df_[df_.label_r == 0]

    # Split non ties per their outcome (left and right)
    df_nonties_left = df_nonties[(df_nonties.label_r == -1)]
    df_nonties_right = df_nonties[(df_nonties.label_r == 1)]

    # Non-ties accuracy
    correct_left = ((df_nonties.label_r == -1) & (df_nonties.rank_left - df_nonties.rank_right > margin)).sum()
    correct_right = ((df_nonties.label_r == 1) & (df_nonties.rank_right - df_nonties.rank_left > margin)).sum()

    nontie_left_accuracy = correct_left / (df_nonties.label_r == -1).sum()
    nontie_right_accuracy = correct_right / (df_nonties.label_r == 1).sum()
    nontie_accuracy = (correct_left + correct_right ) / df_nonties.shape[0]
    
    # Ties accuracy
    tie_accuracy = (abs(df_ties.rank_left - df_ties.rank_right) < margin).sum() / df_ties.shape[0]

    # Overall accuracy
    overall_accuracy = X_test[((df_.label_r == -1) & (df_.rank_left - df_.rank_right > margin)) |
                              ((df_.label_r ==  1) & (df_.rank_right - df_.rank_left > margin)) |
                              ((df_.label_r ==  0) & (abs(df_.rank_left - df_.rank_right) < margin))].shape[0] / df_.shape[0]
    
    return nontie_left_accuracy, nontie_right_accuracy, nontie_accuracy, tie_accuracy, overall_accuracy

In [ ]:
def compute_ranking_accuracy_nomargin(df_,):
    # Split in non-ties and ties
    df_nonties = df_[df_.label_r != 0]

    # Split non ties per their outcome (left and right)
    df_nonties_left = df_nonties[(df_nonties.label_r == -1)]
    df_nonties_right = df_nonties[(df_nonties.label_r == 1)]

    # Non-ties accuracy
    correct_left = ((df_nonties.label_r == -1) & (df_nonties.rank_left - df_nonties.rank_right > 0)).sum()
    correct_right = ((df_nonties.label_r == 1) & (df_nonties.rank_right - df_nonties.rank_left > 0)).sum()

    nontie_left_accuracy = correct_left / (df_nonties.label_r == -1).sum()
    nontie_right_accuracy = correct_right / (df_nonties.label_r == 1).sum()
    nontie_accuracy = (correct_left + correct_right ) / df_nonties.shape[0]

    return nontie_left_accuracy, nontie_right_accuracy, nontie_accuracy

In [ ]:
def compute_ranking_distance(df_):
    # Split in non-ties and ties
    df_nonties = df_[df_.label_r != 0]
    df_ties = df_[df_.label_r == 0]

    # Distance between non-ties
    avg_dist_nonties = abs(df_nonties.rank_left - df_nonties.rank_right).mean()
    
    # Distance between ties
    avg_dist_ties = abs(df_ties.rank_left - df_ties.rank_right).mean()
    
    return avg_dist_nonties, avg_dist_ties

In [ ]:
def compute_classification_accuracy(df_):
    def softmax(x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)
        
    col1_values = df_['logits_l'].values
    col2_values = df_['logits_0'].values
    col3_values = df_['logits_r'].values
        
    probabilities = np.apply_along_axis(softmax, axis=1, arr=np.column_stack((col1_values, col2_values, col3_values)))
    max_indices = np.argmax(probabilities, axis=1)
    # Convert the probabilities back to a DataFrame with appropriate column names
    softmax_df = pd.DataFrame(probabilities, columns=['softmax_logit_l', 'softmax_logit_0', 'softmax_logit_r'])
    max_index_df = pd.DataFrame({'class_predicted': max_indices})
    # Concatenate the new DataFrame with the original DataFrame if needed
    result_df = pd.concat([df.reset_index(drop=True), softmax_df, max_index_df], axis=1,)

    all_accuracy = (result_df.class_predicted == result_df.label_c).sum() / result_df.shape[0]
    tie_accuracy = (result_df[result_df.label_c == 1].class_predicted == result_df[result_df.label_c == 1].label_c).sum() / result_df.shape[0]
    nontie_accuracy = (result_df[result_df.label_c != 1].class_predicted == result_df[result_df.label_c != 1].label_c).sum() / result_df.shape[0]
    
    return all_accuracy, tie_accuracy, nontie_accuracy

### Available results

In [ ]:
model_results = glob('../outputs/saved/synthetic-*.pkl')

In [ ]:
model_data = glob('../data/comparisons_synthetic_pc*.p')

In [ ]:
model_results

In [ ]:
df = pd.read_pickle(model_results[5])

In [ ]:
rank_0 = df[df.rank_right == df.rank_right.min()].image_right.iloc[0]
rank_1 = '/home/mncosta/data/images/01_MS_C_901.jpg'
rank_2 = '/home/mncosta/data/images/01_CP_C_828.jpg'
rank_3 = df[df.rank_right == df.rank_right.max()].image_right.iloc[0]

In [ ]:
fig, axarr = plt.subplots(1,4, figsize=(15, 15));
axarr[0].imshow(mpimg.imread(rank_0))
axarr[0].text(50, 900, '-1.83', style='italic', color='w', fontsize='large', bbox={'facecolor': 'grey', 'alpha': 0.9, 'pad': 1})
axarr[0].set_xticks([]);
axarr[0].set_yticks([]);
axarr[1].imshow(mpimg.imread(rank_1))
axarr[1].text(50, 900, '-1.06', style='italic', color='w', fontsize='large', bbox={'facecolor': 'grey', 'alpha': 0.9, 'pad': 1})
axarr[1].set_xticks([]);
axarr[1].set_yticks([]);
axarr[2].imshow(mpimg.imread(rank_2))
axarr[2].text(50, 900, '0.82', style='italic', color='w', fontsize='large', bbox={'facecolor': 'grey', 'alpha': 0.9, 'pad': 1})
axarr[2].set_xticks([]);
axarr[2].set_yticks([]);
axarr[3].imshow(mpimg.imread(rank_3))
axarr[3].text(50, 900, '1.38', style='italic', color='w', fontsize='large', bbox={'facecolor': 'grey', 'alpha': 0.9, 'pad': 1})
axarr[3].set_xticks([]);
axarr[3].set_yticks([]);